In [8]:
import sys
!{sys.executable} -m pip install opencc-python-reimplemented lxml
!{sys.executable} -m pip list

     |████████████████████████████████| 5.8MB 268kB/s eta 0:00:01
Package                     Version            
--------------------------- -------------------
backcall                    0.1.0              
certifi                     2019.9.11          
decorator                   4.4.1              
ipykernel                   5.1.3              
ipython                     7.9.0              
ipython-genutils            0.2.0              
jedi                        0.15.1             
jupyter-client              5.3.4              
jupyter-core                4.6.1              
lxml                        4.4.1              
opencc-python-reimplemented 0.1.5              
parso                       0.5.1              
pexpect                     4.7.0              
pickleshare                 0.7.5              
pip                         19.3.1             
prompt-toolkit              2.0.10             
ptyprocess                  0.6.0              
Pygments              

In [6]:
!{sys.executable} --version

Python 3.5.2


In [3]:
!which python --version

/usr/bin/python


In [2]:
import sys
!{sys.executable} -m python --


/usr/bin/python3: No module named python


# Load FGC training dataset

In [7]:
import json
file = 'FGC_release_A.json'

with open(file) as f:
    data = json.load(f)

In [1]:
from event_zh.main import sino_extract

In [3]:
sino_extract('event_zh/doc')

Running function sino_extract


/workspace/event_zh/event_zh/SinoCoreferencer


SinoCoreferencer/data/doc 
SinoCoreferencer/data 


/workspace/event_zh/event_zh/SinoCoreferencer/data
/workspace/event_zh


FileNotFoundError: [Errno 2] No such file or directory: 'extract-event'

In [9]:
%%time

from event_zh.main import extract_to_json, extract
from opencc import OpenCC

cc = OpenCC('t2s')
output_dir = 'formosa_ai_outputs'

ee_jsons = {}

def ee(did, t, output_dir):
    
    # convert to simplified chinese
    text = cc.convert(t)
    write_file = did
    with open(write_file, 'w') as f:
        f.write(text)

    # event extraction to json
    ee_json = extract_to_json(write_file, output_dir)
    
    return ee_json

def write_to_mostai(path, ee_json):
    import json
    with open(path, 'w') as f:
        json.dump(ee_json, f)

        
num_data = 2
on_q = False


for i, dic in enumerate(data):
    
    if i >= num_data:
        break
    
    # event extraction on document
    did = dic['DID'][1:]  # strip away 'D' to avoid duplicates
    text = dic['DTEXT']
    try:
        djson = ee(did, text, output_dir)
    

        if on_q:
            # event extraction on questions
            qjsons = []
            qs = dic['QUESTIONS']
            for q in qs:
                text = q['QTEXT']
                qid = q['QID'][1:]
                qjsons.append(ee(qid, text, output_dir))

            # keep all jsons to generate most ai format
            ee_jsons.update({did: djson, 'QUESTIONS': qjsons })
        else:
            ee_jsons.update({did: djson})
    
    except FileNotFoundError:
        raise
            

>>>>> Extracting 001
Running function sino_extract
Call Stanford CoreNLP...
Move Stanford CoreNLP output...
Run entity mention detection...
Run entity typing & subtyping...
Run entity coreference...
Run event trigger identification...
Run event argument identification...
Run event attributes classification...
Run event coreference...
Cleaning...
Congrats! Done!

function sino_extract ran in 9.54s
Running function read_and_output
/workspace/EEtask/Chinese/001: No events!
>>>>> Extracting 002
Running function sino_extract
Call Stanford CoreNLP...
Move Stanford CoreNLP output...
Run entity mention detection...
Run entity typing & subtyping...
Run entity coreference...
Run event trigger identification...
Run event argument identification...
Run event attributes classification...
Run event coreference...
Cleaning...
Congrats! Done!

function sino_extract ran in 9.5s
Running function read_and_output
function read_and_output ran in 0.05s
Finished!


0 D002-S1-EVM0
1
CPU times: user 72 ms, sys: 32 ms, total: 104 ms
Wall time: 19.1 s


In [10]:
write_file = 'FGC_train3.ee.json'

write_to_mostai(write_file, ee_jsons)

In [20]:
write_file = 'FGC_train.ee.json'

write_to_mostai(write_file, ee_jsons)

# Write to MOST AI Format

In [14]:
write_file = 'FGC_sample5.ee.json'
def write_to_mostai(path, ee_json):
    import json
    with open(path, 'w') as f:
        json.dump(ee_json, f)

write_to_mostai(write_file, ee_jsons)

# Single Text

In [1]:
text1 = "梵語在社交中口頭使用，並且在早期古典梵語文獻的發展中維持口頭傳統。在印度，書寫形式是當梵語發展成俗語之後才出現的；在書寫梵語的時候，書寫系統的選擇受抄寫者所處地域的影響。同樣的，所有南亞的主要書寫系統事實上都用於梵語文稿的抄寫。自19世紀晚期，天城文被定為梵語的標準書寫系統，十分可能的原因是歐洲人有用這種文字印刷梵語文本的習慣。最早的已知梵語碑刻可確定為公元前一世紀。它們採用了最初用於俗語而非梵語的婆羅米文。第一個書寫梵語的證據，出現在晚於它的俗語的書寫證據之後的幾個世紀，這被描述為一種悖論。在梵語被書寫下來的時候，它首先用於行政、文學或科學類的文本。宗教文本口頭傳承，在相當晚的時候才「不情願」地被書寫下來。"
text2 = "日本第二大都會區大阪都市圈發生芮氏規模6.1，最大震度6弱的強烈地震，造成至少4人死亡及376人受傷。震央位於大阪府北部，震源深度僅13公里，屬極淺層的「直下型地震」。"
text3 = "不多一會，朗山端了一瓦缽子飯來了，只看那蓋子縫裡，熱氣向外亂噴，那種白米飯的香味，直鑽到人家鼻子眼裡去。雖是已經吃了兩個饅頭，肚子裡有點東西了，可是聞到這種香氣，更引起胃欲。只見劉朗山將缽子蓋一掀，看到裡面鬆鬆的半缽飯，其白如雪，恨不得將瓦缽端了過來，一人獨吞下去，現在瓦缽子在劉朗山手裡，爭奪不得，便望了飯笑道：「這飯兩個人吃，怕是不夠吧？」朗山點著頭道：「我本來打算煮一餐飯作兩餐吃的，怎樣會不夠？」於是在床底下網籃裡取出兩隻飯碗，盛了飯放在桌上。他因自己一雙筷子被士毅佔了，由網籃裡找到桌子抽屜裡，更由桌子抽屜裡，找到書堆裡，為了一雙筷子，找了許久的工夫。士毅在人家主人翁未曾來吃的時候，又不便先吃，只好瞪了兩隻眼睛，望著這一大碗白米飯發呆，好容易把筷子找來，才開始吃飯，士毅便是不吃菜，這飯爬到口裡去，也就香甜可口，三下兩下，把一碗飯就吃了下去。"
text4 = "(台北)台北市南港區.近日發生;多部車輛遭歹徒以瓦斯槍.隨機射擊毀損的案件;案發後.南港警分局積極蒐證偵辦,最後在案發48小時內,順利逮捕.曾有類似前科的兩名犯嫌!記者靳傑報導台北市警局南港分局21號晚間獲報:轄區捷運昆陽機場附近,有6部自小客貨車,遭歹徒以不明槍械隨機射擊毀損,於是出動鑑識人員,和轄區同德派出所員警.到場蒐證;現場並發現遺留的鋼珠,和.BB彈等證物;經訪查及蒐集案發地相關跡證,發現ㄧ部白色疑似作案車輛,於是報請士檢指揮偵辦,23號清晨.分別在轄區南港路3段,和.新北市汐止中興路,陳姓及劉姓犯嫌住處,將兩人逮捕;兩名男子落網後辯稱:因為心情不佳,才持瓦斯槍隨機掃射路邊停放車輛;南港警分局偵查隊長.顏光燦說~VOICE台北市警局南港分局初步調查:46歲的劉姓和19歲的陳姓犯嫌,分別有竊盜.毒品等前科,平日還充當藥頭提供毒品;據了解,兩人4月初曾以相同手法在轄區福德街一帶作案,被依照毀損及竊盜犯嫌送辦,但仍不知悔改;訊問後.已依照毀損罪嫌,移送士林地檢署偵辦!"
text5 = "自稱朱元璋後代的台北光信里里長朱雪璋，涉嫌相約比武時設局砍人，造成對手腳筋斷裂，台北地院今天依重傷害罪將朱雪璋判處六年刑期，他的妻子和二十多名共犯則判處2到6月不等刑期。記者呂明宗報導擁有空手道黑帶五段身手的朱雪璋，前年在網路上與被稱為深海閻王的王姓武術家相約在自家道館比武，王姓男子如期赴約，但遭十多人持棍棒毆傷，還被砍斷腳筋一度命危，案發後，朱雪璋辯稱是對方帶刀赴約，他為了保護家人才反擊自衛，不過檢警深入追查發現，王雪璋縝密規劃犯案，不僅透過幫派份子召集人馬埋伏，還要求員工遮蔽道館監視器，依重傷害等罪嫌將朱雪璋等26人起訴。台北地院審理時，朱雪璋雖然坦承傷人，但否認有重傷害犯意，不過，合議庭認定朱雪璋動手傷人造成對手重傷害，判處六年，但原本被認定是重傷害共犯的妻子則獲得輕判，台北地院行政庭長廖建瑜說—tape至於其他共犯則分別被依傷害和幫助傷害等罪判處2個月到6個月不等刑期，都可易科罰金。被害人深海閻王聆判後認為判的太輕，也不解他的妻子可以從重罪改判輕罪。"
# 成立
text6 = "為配合政府三階段電動車推動政策，並朝向轉型為新能源企業暨綠能事業發展，台灣中油董事長戴謙於今年2月率團前往日本考察先進新能源科技及尋求未來可能的合作夥伴，其中包括日本TDK。台灣中油表示，TDK創始於1935年，成立迄今達83年，過去以生產電子零件相關產品為主，從錄音卡帶至硬碟讀取機，全球市占率第一，近年來意識到綠能潮流，轉向研發電池與電池芯技術，未來將朝感應器及powersolution發展，堪稱轉型前輩，值得台灣中油借鏡。" 
text7 = "臺北市長柯文哲明天(7月1號)將赴上海參加雙城論壇，他在接受媒體聯訪時表示，雙城論壇一開始設定目標就很清楚，希望兩岸僵局在對等尊嚴的原則下獲得轉圜。記者劉碧薇報導------臺北上海雙城論壇7月2號在上海舉行，論壇主題為「健康城市」，臺北市柯文哲將在7月1號抵達，外界關注，柯文哲這次前往大陸，傳出陸方擬好腳本，希望柯能闡述兩岸之間不是國與國的關係？對此，柯文哲受訪時表示，「大陸有大陸的想法，我們有我們的堅持」既然雙方定義是兩岸城市交流，就減少談政治高層的問題。他強調，雙城論壇一開始設定目標就很清楚------台北、上海雙城論壇7月2日將在上海東方濱江大酒店舉行，柯文哲與57名市府人員、10名台北市議員1號下午將搭機啟程，預計7月3日晚間回台。依北市府日前公布的參訪行程，包括第一天在上海第六人民醫院與青年醫生及醫學生午餐交流，餐後觀摩骨科手術、參訪徐匯區斜土街道江南新村居民區參觀徐匯濱江規劃館，以及高中生街舞交流活動。2號晚間由北市府晚宴，邀請陸方參加；3號參訪長江隧橋管控中心、崇明區社區、東灘島濕地和長江口中華鱘保護區等。"
text = text7

In [3]:
from pprint import pprint
pprint(ee_json)

NameError: name 'ee_json' is not defined

In [21]:
import json
s = json.load()


AttributeError: 'str' object has no attribute 'read'

In [19]:
%%time
ee_json = extract_to_json('input.file', 'formosa_ai_outputs')

ee_json

In [36]:
with open('event_sample_output.json', 'w') as f:
    json.dump(out_json, f, ensure_ascii=False, indent=4)

In [37]:
from pprint import pprint
pprint(out_json)


defaultdict(<class 'dict'>,
            {'9_410': {'Passage': {'D9_410P-S1-EVM0': {'args': [{'char_b': 0,
                                                                 'char_e': 2,
                                                                 'in_tokens': [['台北']],
                                                                 'role': 'Destination',
                                                                 'text': '台北',
                                                                 'token_b': 0,
                                                                 'token_e': 1},
                                                                {'char_b': 17,
                                                                 'char_e': 19,
                                                                 'in_tokens': [['上海']],
                                                                 'role': 'Destination',
                                                                 'te